In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import onnx
import onnxruntime
import torchvision.transforms as transforms
from torchvision import datasets

In [ ]:
train_data = datasets.MNIST('..', train=True, download=True)
test_data = datasets.MNIST('..', train=False, download=True)

In [ ]:
# plot image
def mnist_plt(img):
    if len(img) == 2:
        label = img[1]
        img = img[0]
        print(f"label: {label}")
        
    img = np.matrix(img)
    plt.imshow(img, cmap="gray")
    plt.axis('off')
    plt.show()

def ascii_plot(img, char=" ", outer="X"):
    for row in img:
        for pixel in row:
            if pixel > .5:
                print(char, end="")
            else:
                print(outer, end="")
        print("")

In [ ]:
model_path = "C:/workspace/tenseal-inference/models/lenet-5_square.onnx"

In [ ]:
def softmax(x):
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x

def comparison(model_path, x, tenseal_result_path, plot=True):
    if plot:
        mnist_plt(x)
    model = onnx.load(model_path)
    session = onnxruntime.InferenceSession(model_path)
    
    y = session.run(None, {model.graph.input[0].name: x})[0][0]
    
    sy = softmax(y)
    
    print(f"cleartext onnxruntime: {np.argmax(y)}")
    for i,yi in enumerate(y):
        print(f"{i} {yi:+10.6f} {sy[i]:.4f}")
    print()

    ## tenseal-inference result
    with open(tenseal_result_path, 'rb') as f:
        a = np.load(f)

    sa = softmax(a)
        
    print(f"tenseal-inference: {np.argmax(a)}")
    for i,ai in enumerate(a):
        print(i, f"{ai:+#10.6f} {sa[i]:.4f}")
    print()

    # plt.bar(np.arange(10)-0.2, y, width=.4, label="clear")
    # plt.bar(np.arange(10)+0.2, a, width=.4, label="tenseal")
    # plt.grid()
    # plt.legend()
    # plt.show()

In [ ]:
index = 0

data = test_data[index]
x = np.array(data[0], dtype=np.float32).reshape(1,1,28,28) / 255
tenseal_result_path = f"../tenseal-inference/tmp/poster/result{index}.npy"

print(f"Correct result: {data[1]}")
comparison(model_path, x, tenseal_result_path)